In [103]:
import pandas as pd
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [104]:
data = pd.read_excel('_cotacoes_completa_diario.xlsx')
filter_col = [col for col in data if col.startswith('close')]
dados = data[filter_col]

In [105]:
dados.head()

,close IBOV,close ABEV3,close B3SA3,close BBAS3,close BBDC3,close BBDC4,close BRAP4,close BRFS3,close BRKM5,close BRML3,...,close RENT3,close SBSP3,close SUZB3,close TIMP3,close UGPA3,close USIM5,close VALE3,close VIVT4,close WEGE3,close YDUQ3
0,63394,8.01,7.80,16.48,8.25,9.58,25.91,24.43,15.97,10.28,...,6.94,12.32,9.64,7.20,11.35,13.06,35.87,24.98,4.38,5.14
1,63891,7.99,7.75,16.66,8.30,9.73,26.23,24.39,15.81,10.10,...,6.89,12.40,9.73,7.10,11.53,13.35,36.32,24.88,4.34,5.11
2,63039,8.07,7.57,16.58,8.32,9.69,25.85,24.34,15.63,10.12,...,6.71,12.54,9.64,7.09,11.43,13.13,35.60,24.34,4.41,5.20
3,62565,8.04,7.51,16.44,8.20,9.57,25.79,23.44,15.38,9.90,...,6.63,12.28,9.31,7.14,11.40,12.96,35.48,24.28,4.58,5.14
4,62207,7.94,7.62,16.00,8.07,9.44,25.95,23.51,15.09,9.85,...,6.79,12.44,9.51,7.19,11.20,12.63,35.70,24.25,4.44,5.19


In [106]:
# dados = dados.pct_change()/
dados['IBOV'] = data['close IBOV']
shift = dados['close IBOV'].shift(1)
dados['dependente'] = shift


<ipython-input-106-2d991749cfcf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['IBOV'] = data['close IBOV']
<ipython-input-106-2d991749cfcf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['dependente'] = shift


In [107]:
dados = dados.dropna()

In [108]:
training_data_len = math.ceil(len(dados)*.8)

In [109]:
dados.head()

,close IBOV,close ABEV3,close B3SA3,close BBAS3,close BBDC3,close BBDC4,close BRAP4,close BRFS3,close BRKM5,close BRML3,...,close SUZB3,close TIMP3,close UGPA3,close USIM5,close VALE3,close VIVT4,close WEGE3,close YDUQ3,IBOV,dependente
1,63891,7.99,7.75,16.66,8.30,9.73,26.23,24.39,15.81,10.10,...,9.73,7.10,11.53,13.35,36.32,24.88,4.34,5.11,63891,63394.0
2,63039,8.07,7.57,16.58,8.32,9.69,25.85,24.34,15.63,10.12,...,9.64,7.09,11.43,13.13,35.60,24.34,4.41,5.20,63039,63891.0
3,62565,8.04,7.51,16.44,8.20,9.57,25.79,23.44,15.38,9.90,...,9.31,7.14,11.40,12.96,35.48,24.28,4.58,5.14,62565,63039.0
4,62207,7.94,7.62,16.00,8.07,9.44,25.95,23.51,15.09,9.85,...,9.51,7.19,11.20,12.63,35.70,24.25,4.44,5.19,62207,62565.0
5,61513,7.81,7.50,15.56,8.00,9.28,25.62,23.12,14.55,9.96,...,9.36,7.38,11.12,12.62,35.46,23.93,4.48,5.16,61513,62207.0


In [110]:
#selecionando variavel de interesse
Y = dados['dependente']
#selecionando os dados com base nas features
X = dados.drop(['IBOV', 'dependente'], axis=1)

#separa os dados em treino e validação

In [111]:
X_treino = X[0:training_data_len]
Y_treino = Y[0:training_data_len]
X_validacao = X[training_data_len:]
Y_validacao = Y[training_data_len:]


In [112]:
modelo_randomforest = RandomForestRegressor(random_state=1,n_jobs=6,n_estimators=2000,min_samples_split=5,min_samples_leaf=2,max_features='auto',max_depth=50,bootstrap=True)
modelo_randomforest.fit(X_treino,Y_treino)

RandomForestRegressor(max_depth=50, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=2000, n_jobs=6, random_state=1)

In [113]:
rf_predict = modelo_randomforest.predict(X_validacao)


In [114]:
rf_mae = mean_absolute_error(rf_predict, Y_validacao)
print(rf_mae)


11938.281445147715


In [115]:
len(rf_predict)

432

In [116]:
from sklearn.ensemble import GradientBoostingRegressor
#criando o modelo tunadão 
gbmodel = GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')
#treinando o modelo
gbmodel.fit(X_treino, Y_treino)
#dando o predict no valor da casa com base nos dados
resultado_gb = gbmodel.predict(X_validacao)
#resultado utilizando a funcao score
print(f'Resultado utilizando a funcao score do sklear: {gbmodel.score(X_validacao,Y_validacao):.2f}')
#resultado utilizando a funcao evaluate
# base_accuracy = evaluate(gbmodel, X_validacao, Y_validacao)


Resultado utilizando a funcao score do sklear: -0.81


In [117]:
rf_mae = mean_absolute_error(resultado_gb, Y_validacao)
print(rf_mae)

13303.542166894584


In [118]:
#0.014308571219983676, 0.013011246510589268


In [ ]:
def ind_for(dados, periodo): #força
    pos = 0 #posicao
    tempo = periodo
    desempenho = []
    comprado = False
    #valor de compra, venda e resultado
    c = 0
    v = 0
    res = 0
    for i in range(len(dados)):
        rsi_value = (dados.rsi[dados.index==i].values[0])
        if comprado:
            tempo -= 1

        if rsi_value <= 25: #compra de acordo com o rsi
            if not comprado:
                comprado = True
                tempo = periodo
                c += dados['close IBOV'][dados.index ==i].values[0]

        if comprado == True and tempo == 0:
            comprado = False
            v += dados['close IBOV'][dados.index ==i].values[0]
            res = c-v
            c,v = 0,0

        if res not in desempenho:
            desempenho.append(res)
            
    return np.mean(desempenho)
        